# Kaggle Titanic Machine Learning
- source of competition: https://www.kaggle.com/c/titanic
- Data Dictionary: https://www.kaggle.com/c/titanic/data
- useful link for saving to GitHub: https://www.kaggle.com/questions-and-answers/72234

In [2]:
# Importing libraries
%matplotlib inline
import numpy as np 
import pandas as pd 
import pandas_profiling

# Setting Random Seed For Reproducibility
import random
random.seed(123)

# Displaying Max rows
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)

# Listing Files
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv


In [3]:
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv') # for final evaluation/submission only

In [4]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# Data Wrangling/Cleaning

In [6]:
# Creating train/val/test split prior to transformations (avoid data leakage)

X = df_train.drop(['Survived'],axis = 1)
y = df_train.Survived

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.15, random_state = 3) # test set 15% train
X_train, X_val, y_train, y_val   = train_test_split(X_train, y_train, test_size = 0.15, random_state = 3) #validation set 15% train

In [7]:
for i in [X_train,X_test,X_val]:
    print(i.shape)

(643, 11)
(134, 11)
(114, 11)


# Exploratory Data Analysis

General thoughts based on the profile below
- PassengerId - removing due to ID variable
- Missing values: Age, Cabin, Fare, Embarked
- Correlations in Fare-Class-Age

In [8]:
#combining the train feature/target data for EDA/Data Wrangling

df_train_split = pd.concat([X_train, y_train], axis = 1)
df_train_split.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
387,388,2,"Buss, Miss. Kate",female,36.0,0,0,27849,13.0000,NaN,S,1
531,532,3,"Toufik, Mr. Nakli",male,NaN,0,0,2641,7.2292,NaN,C,0


In [9]:
#making use of the profile package for EDA plots/stats/...

profile = pandas_profiling.ProfileReport(df_train_split, title = "EDA Profile Train Data Report")

In [10]:
profile.to_widgets()

# Data Wrangling and Feature Engineering
- only on training dataset, will use a pipeline for val/test and final submission test set

In [11]:
# Missing Values Handling
print(df_train_split.Embarked.value_counts())

#Embarked only 1 missing, fill with most common of S, C, Q (will be S)
df_train_split.Embarked = df_train_split.Embarked.fillna(df_train_split.Embarked.value_counts().index[0]) #using value_counts top record

S    475
C    118
Q     49
Name: Embarked, dtype: int64


In [12]:
# dropping passenger id (is an id)
df_train_split.drop(['PassengerId'], axis = 1, inplace = True)

In [13]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(categories='auto')
feature_array = ohe.fit_transform(df_train_split[['Parch','Pclass','Sex','SibSp','Embarked']]).toarray()
#feature_labels = ohe.categories_

In [14]:
features = pd.DataFrame(feature_array, columns=ohe.get_feature_names())
print(features.shape)
features.head()

(643, 22)


,x0_0,x0_1,x0_2,x0_3,x0_4,x0_5,x0_6,x1_1,x1_2,x1_3,x2_female,x2_male,x3_0,x3_1,x3_2,x3_3,x3_4,x3_5,x3_8,x4_C,x4_Q,x4_S
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [15]:
df_train_split = df_train_split.drop(['Parch','Pclass','Sex','SibSp','Embarked'], axis = 1)


In [16]:
print(df_train_split.shape)
df_train_split.head()

(643, 6)


,Name,Age,Ticket,Fare,Cabin,Survived
387,"Buss, Miss. Kate",36.0,27849,13.0000,NaN,1
531,"Toufik, Mr. Nakli",NaN,2641,7.2292,NaN,0
480,"Goodwin, Master. Harold Victor",9.0,CA 2144,46.9000,NaN,0
217,"Jacobsohn, Mr. Sidney Samuel",42.0,243847,27.0000,NaN,0
799,"Van Impe, Mrs. Jean Baptiste (Rosalie Paula Go...",30.0,345773,24.1500,NaN,0


In [17]:
df_train_split = pd.concat([df_train_split.reset_index(drop=True),features.reset_index(drop=True)], axis = 1)

In [18]:
df_train_split.shape
df_train_split.head()

,Name,Age,Ticket,Fare,Cabin,Survived,x0_0,x0_1,x0_2,x0_3,x0_4,x0_5,x0_6,x1_1,x1_2,x1_3,x2_female,x2_male,x3_0,x3_1,x3_2,x3_3,x3_4,x3_5,x3_8,x4_C,x4_Q,x4_S
0,"Buss, Miss. Kate",36.0,27849,13.0000,NaN,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,"Toufik, Mr. Nakli",NaN,2641,7.2292,NaN,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,"Goodwin, Master. Harold Victor",9.0,CA 2144,46.9000,NaN,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,"Jacobsohn, Mr. Sidney Samuel",42.0,243847,27.0000,NaN,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,"Van Impe, Mrs. Jean Baptiste (Rosalie Paula Go...",30.0,345773,24.1500,NaN,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [19]:
df_train_split.head()

,Name,Age,Ticket,Fare,Cabin,Survived,x0_0,x0_1,x0_2,x0_3,x0_4,x0_5,x0_6,x1_1,x1_2,x1_3,x2_female,x2_male,x3_0,x3_1,x3_2,x3_3,x3_4,x3_5,x3_8,x4_C,x4_Q,x4_S
0,"Buss, Miss. Kate",36.0,27849,13.0000,NaN,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,"Toufik, Mr. Nakli",NaN,2641,7.2292,NaN,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,"Goodwin, Master. Harold Victor",9.0,CA 2144,46.9000,NaN,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,"Jacobsohn, Mr. Sidney Samuel",42.0,243847,27.0000,NaN,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,"Van Impe, Mrs. Jean Baptiste (Rosalie Paula Go...",30.0,345773,24.1500,NaN,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [20]:
df_train_split.shape

(643, 28)

In [21]:
# # Encoding categorical features (will be used to impute Age/Cabin missing values as possible)

# df_train_split = pd.get_dummies(df_train_split, columns=['Parch','Pclass','Sex','SibSp','Embarked'])
# df_train_split

In [22]:
# Encoding the Age Missing Values that are 'S' with the training data median Age 
median_age_train = df_train_split.Age.median()
df_train_split['Age'] = df_train_split['Age'].apply(lambda x : median_age_train if pd.isnull(x) else x)
df_train_split.Age.isna().sum()

0

### Cabin Missing Values

In [23]:
#INPROGRESS #Missing Values Cabin - taking the initial value
df_train_split.Cabin = df_train_split[['Cabin']].fillna(value= 'Z')
df_train_split['Cabin_augment'] = df_train_split.Cabin.apply(lambda x : x[0]) # augmenting dataset, only want the first letter (numbers not matter)
df_train_split.Cabin_augment.value_counts()

Z    491
C     44
B     33
D     26
E     24
A     12
F     11
T      1
G      1
Name: Cabin_augment, dtype: int64

In [24]:
df_train_split[['Cabin_augment','Fare']].groupby(['Cabin_augment']).mean().round(2)

,Fare
Cabin_augment,
A,39.35
B,118.79
C,91.22
D,54.26
E,44.00
F,20.22
G,16.70
T,35.50
Z,18.96


In [25]:
from sklearn.preprocessing import OneHotEncoder

ohe_Cabin_augment = OneHotEncoder(categories='auto')
feature_array_Cabin_augment = ohe_Cabin_augment.fit_transform(df_train_split[['Cabin_augment']]).toarray()
#feature_labels = ohe.categories_
features_Cabin_augment = pd.DataFrame(feature_array_Cabin_augment, columns=ohe_Cabin_augment.get_feature_names())


In [26]:
df_train_split.drop(['Cabin','Cabin_augment'], axis =1, inplace = True)
df_train_split = pd.concat([df_train_split,features_Cabin_augment], axis = 1)

In [27]:
df_train_split.head()

,Name,Age,Ticket,Fare,Survived,x0_0,x0_1,x0_2,x0_3,x0_4,x0_5,x0_6,x1_1,x1_2,x1_3,x2_female,x2_male,x3_0,x3_1,x3_2,x3_3,x3_4,x3_5,x3_8,x4_C,x4_Q,x4_S,x0_A,x0_B,x0_C,x0_D,x0_E,x0_F,x0_G,x0_T,x0_Z
0,"Buss, Miss. Kate",36.0,27849,13.0000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,"Toufik, Mr. Nakli",29.0,2641,7.2292,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,"Goodwin, Master. Harold Victor",9.0,CA 2144,46.9000,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,"Jacobsohn, Mr. Sidney Samuel",42.0,243847,27.0000,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,"Van Impe, Mrs. Jean Baptiste (Rosalie Paula Go...",30.0,345773,24.1500,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [28]:
# df_train_split = pd.get_dummies(df_train_split, columns=['Cabin_augment'])

In [29]:
# df_train_split.head()

In [30]:
df_train_split.drop(['Name','Ticket'], axis = 1, inplace = True)

In [31]:
df_train_split.head()

,Age,Fare,Survived,x0_0,x0_1,x0_2,x0_3,x0_4,x0_5,x0_6,x1_1,x1_2,x1_3,x2_female,x2_male,x3_0,x3_1,x3_2,x3_3,x3_4,x3_5,x3_8,x4_C,x4_Q,x4_S,x0_A,x0_B,x0_C,x0_D,x0_E,x0_F,x0_G,x0_T,x0_Z
0,36.0,13.0000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,29.0,7.2292,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,9.0,46.9000,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,42.0,27.0000,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,30.0,24.1500,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [32]:
# checking that all missing values are taken care of
print(df_train_split.isna().sum().sum())
df_train_split.shape

0


(643, 34)

In [33]:
df_train_split_X = df_train_split.drop(['Survived'],axis = 1)
df_train_split_y = df_train_split[['Survived']]

In [34]:
# Performing Save Data Wrangling Steps on the Val/Test Data

#combining the train feature/target data for EDA/Data Wrangling
df_val_split = pd.concat([X_val, y_val], axis = 1)
df_val_split.Embarked = df_val_split.Embarked.fillna('S')
df_val_split.drop(['PassengerId'], axis = 1, inplace = True)

feature_array = ohe.transform(df_val_split[['Parch','Pclass','Sex','SibSp','Embarked']]).toarray()
features = pd.DataFrame(feature_array, columns=ohe.get_feature_names())
df_val_split = df_val_split.drop(['Parch','Pclass','Sex','SibSp','Embarked'], axis = 1)
df_val_split = pd.concat([df_val_split.reset_index(drop=True),features.reset_index(drop=True)], axis = 1)

df_val_split['Age'] = df_val_split['Age'].apply(lambda x : median_age_train if pd.isnull(x) else x)

df_val_split.Cabin = df_val_split[['Cabin']].fillna(value= 'Z')
df_val_split['Cabin_augment'] = df_val_split.Cabin.apply(lambda x : x[0])

feature_array_Cabin_augment = ohe_Cabin_augment.transform(df_val_split[['Cabin_augment']]).toarray()
features_Cabin_augment = pd.DataFrame(feature_array_Cabin_augment, columns=ohe_Cabin_augment.get_feature_names())

df_val_split.drop(['Cabin','Cabin_augment'], axis =1, inplace = True)
df_val_split = pd.concat([df_val_split,features_Cabin_augment], axis = 1)

df_val_split.drop(['Name','Ticket'], axis = 1, inplace = True)

print(df_val_split.shape)
df_val_split.head()

(114, 34)


,Age,Fare,Survived,x0_0,x0_1,x0_2,x0_3,x0_4,x0_5,x0_6,x1_1,x1_2,x1_3,x2_female,x2_male,x3_0,x3_1,x3_2,x3_3,x3_4,x3_5,x3_8,x4_C,x4_Q,x4_S,x0_A,x0_B,x0_C,x0_D,x0_E,x0_F,x0_G,x0_T,x0_Z
0,14.0,46.9000,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,36.0,10.5000,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,9.0,31.3875,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,19.0,7.8792,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,45.0,8.0500,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [35]:
df_val_split_X = df_val_split.drop(['Survived'],axis = 1)
df_val_split_y = df_val_split[['Survived']]

# Model Developement 

In [37]:
# Baseline Model 
import xgboost as xgb

eval_set = [(df_val_split_X,df_val_split_y)]

model_xgb = xgb.XGBClassifier(learning_rate = 0.01)
model_xgb.fit(df_train_split_X, df_train_split_y, early_stopping_rounds=10, eval_metric="error", eval_set= eval_set,verbose = 0)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [38]:
from sklearn.metrics import accuracy_score

print("Training Accuracy:", accuracy_score(model_xgb.predict(df_train_split_X),df_train_split_y))
print("Validation Accuracy:", accuracy_score(model_xgb.predict(df_val_split_X),df_val_split_y))

Training Accuracy: 0.880248833592535
Validation Accuracy: 0.7894736842105263


In [40]:
# Adding Parameter Tuning
from sklearn.model_selection import GridSearchCV

param_grid = {
    "learning_rate": [0.1,0.05],
    'max_depth': [2,3,4,5,6],
    'min_child_weight': [1, 2,4,6,8,10],
    'subsample': [0.5, 0.7, 0.9],
    'n_estimators': [5, 30, 100, 250, 500],
}

grid_clf = GridSearchCV(xgb.XGBClassifier() , param_grid, scoring='accuracy', cv=None)
grid_clf.fit(df_train_split_X, df_train_split_y , early_stopping_rounds=10 , eval_metric="error", eval_set= eval_set,verbose = False)

best_parameters = grid_clf.best_params_

print('Grid Search found the following optimal parameters: ')
for param_name in sorted(best_parameters.keys()):
    print('%s: %r' % (param_name, best_parameters[param_name]))
    
print("Training Accuracy:", accuracy_score(grid_clf.predict(df_train_split_X),df_train_split_y))
print("Validation Accuracy:", accuracy_score(grid_clf.predict(df_val_split_X),df_val_split_y))


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Grid Search found the following optimal parameters: 
learning_rate: 0.1
max_depth: 4
min_child_weight: 4
n_estimators: 30
subsample: 0.9
Training Accuracy: 0.8227060653188181
Validation Accuracy: 0.7894736842105263


In [53]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

gnb.fit(df_train_split_X, df_train_split_y.values.ravel())

print("Training Accuracy:", accuracy_score(gnb.predict(df_train_split_X),df_train_split_y))
print("Validation Accuracy:", accuracy_score(gnb.predict(df_val_split_X),df_val_split_y))

Training Accuracy: 0.4307931570762053
Validation Accuracy: 0.4649122807017544


In [52]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression(random_state=0, max_iter = 1000).fit(df_train_split_X, df_train_split_y.values.ravel())

print("Training Accuracy:", accuracy_score(clf_log.predict(df_train_split_X),df_train_split_y))
print("Validation Accuracy:", accuracy_score(clf_log.predict(df_val_split_X),df_val_split_y))

Training Accuracy: 0.8289269051321928
Validation Accuracy: 0.7807017543859649


# Potential Next Steps / Changes to Consider
- Potentially use K-Fold Cross validation due to small size 
- https://alexforrest.github.io/you-might-be-leaking-data-even-if-you-cross-validate.html
- https://machinelearningmastery.com/data-preparation-without-data-leakage/